### Transfer Learning with ResNet

In [1]:
import os
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time
import torchvision.models as models
from matplotlib import pyplot as plt

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
image_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
dataset_path = "./dataset"

dataset = datasets.ImageFolder(root=dataset_path, transform=image_transforms)
len(dataset)

2300

In [5]:
class_names = dataset.classes
class_names 

['F_Breakage', 'F_Crushed', 'F_Normal', 'R_Breakage', 'R_Crushed', 'R_Normal']

In [6]:
num_classes = len(dataset.classes)
num_classes

6

In [7]:
train_size = int(0.75*len(dataset))
val_size = len(dataset) - train_size

train_size, val_size

(1725, 575)

In [8]:
from torch.utils.data import random_split

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [9]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

In [10]:
# Load the pre-trained ResNet model
class CarClassifierResNet(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.5):
        super().__init__()
        self.model = models.resnet50(weights='DEFAULT')
        # Freeze all layers except the final fully connected layer
        for param in self.model.parameters():
            param.requires_grad = False
            
        # Unfreeze layer4 and fc layers
        for param in self.model.layer4.parameters():
            param.requires_grad = True            
            
        # Replace the final fully connected layer
        self.model.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(self.model.fc.in_features, num_classes)
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [11]:
# Define the objective function for Optuna
def objective(trial):
    # Suggest values for the hyperparameters
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.7)
    
    # Load the model
    model = CarClassifierResNet(num_classes=num_classes, dropout_rate=dropout_rate).to(device)
    
    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
    
    # Training loop (using fewer epochs for faster hyperparameter tuning)
    epochs = 3
    start = time.time()
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for batch_num, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        
        # Validation loop
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        
        # Report intermediate result to Optuna
        trial.report(accuracy, epoch)
        
        # Handle pruning (if applicable)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    end = time.time()
    print(f"Execution time: {end - start} seconds")
    
    return accuracy

In [12]:
# Create the study and optimize
import optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

[I 2025-02-27 17:06:21,368] A new study created in memory with name: no-name-b173fc58-9270-4147-9487-f50f28f6f1fd
[I 2025-02-27 17:30:02,765] Trial 0 finished with value: 49.91304347826087 and parameters: {'lr': 2.0158446091000113e-05, 'dropout_rate': 0.6552433045741607}. Best is trial 0 with value: 49.91304347826087.


Execution time: 1420.779471874237 seconds


[I 2025-02-27 17:52:40,250] Trial 1 finished with value: 72.69565217391305 and parameters: {'lr': 0.002738090675420578, 'dropout_rate': 0.5120458232576381}. Best is trial 1 with value: 72.69565217391305.


Execution time: 1356.432828426361 seconds


[I 2025-02-27 18:15:31,898] Trial 2 finished with value: 74.26086956521739 and parameters: {'lr': 0.0058828411343760136, 'dropout_rate': 0.36566581365302}. Best is trial 2 with value: 74.26086956521739.


...

[I 2025-02-27 19:09:42,786] Trial 5 finished with value: 74.95652173913044 and parameters: {'lr': 0.00048043527914101985, 'dropout_rate': 0.4764865185670926}. Best is trial 4 with value: 78.78260869565217.


Execution time: 974.4304718971252 seconds


[I 2025-02-27 19:14:40,073] Trial 6 pruned. 
[I 2025-02-27 19:36:07,972] Trial 7 finished with value: 76.69565217391305 and parameters: {'lr': 0.0012662058914162794, 'dropout_rate': 0.46732111528893866}. Best is trial 4 with value: 78.78260869565217.


Execution time: 1287.5005824565887 seconds


[I 2025-02-27 20:13:27,036] Trial 8 pruned. 
[I 2025-02-27 20:39:20,955] Trial 9 pruned. 


In [13]:
study.best_params

{'lr': 0.0013017355110099633, 'dropout_rate': 0.5240949625392353}